In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import sys
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import warnings
warnings.filterwarnings("ignore")
import torchvision
from torch.nn import init
import math
from torch.autograd import Variable

In [0]:
class BasicBlock(nn.Module):
  '''
  Basic repeating Block of a fully preactivated Resnet architecture.
  
  arguments:
    in_channels:no. of input channels
    out_channels: no. of output channels
    drop: dropout probability
    stride: stride for the convnet(default value 1)
  
  returns:
    architecture structure
  '''
  def __init__(self,in_channels,out_channels,drop,stride=1):
    
    super(BasicBlock, self).__init__()
    self.bn1 = nn.BatchNorm2d(in_channels)
    self.conv1 = nn.Conv2d(in_channels,out_channels,
                           kernel_size=3,stride=stride,padding=1,bias=False)
    #dropout automatically shuts down during eval mode
    self.drop = nn.Dropout(drop,inplace=True)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(in_channels=out_channels,out_channels=out_channels,
                           kernel_size=3,stride=1,padding=1,bias=False)
    self.relu = nn.ReLU(inplace=True)
    
    if in_channels!=out_channels or stride!=1:
      self.identity = nn.Sequential(nn.Conv2d(in_channels=in_channels,out_channels=out_channels,
                                             kernel_size=1,stride=stride,bias=False))
    else:
      self.identity = lambda x: x      
    
  def forward(self,x):
    out = self.conv1(self.relu(self.bn1(x)))
    #out = self.drop(out)
    out = self.conv2(self.relu(self.bn2(out)))
    i = self.identity(x)
    out += i
    return out
      

In [0]:
class Resnet(nn.Module):
  '''
  Main module of the Resnet Architecture modified for tiny-imagenet.
  
  arguments:
    block: BasicBlock object
    num_blocks: 3 element list containing no. of blocks per stage
    num_classes: number of classes(default 200)
    
   returns:
     architecture structure
  '''
  
  def __init__(self,block,num_blocks,num_classes=200):
    super(Resnet, self).__init__()
    #changed stride 2 for tiny imagenet
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=2,padding=1,bias=False)
    self.in_channels = 16
    self.stage1 = self.create_stage(block,16,num_blocks[0],stride=1)
    self.stage2 = self.create_stage(block,32,num_blocks[1],stride=2)
    self.stage3 = self.create_stage(block,64,num_blocks[2],stride=2)
    #self.lastact = nn.Sequential(nn.BatchNorm2d(64),nn.ReLU(inplace=True))
    #replaced avg pool with adaptive pool
    self.avgPool = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(64,num_classes)
    
    #weights and biases initialisations for differnt parts of the network
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        #m.bias.data.zero_()
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        init.kaiming_normal(m.weight)
        m.bias.data.zero_()
    
  def create_stage(self,block,out_channels,num_blocks,stride=1):
    layers = []
    layers.append(block(self.in_channels,out_channels,0.3,stride))
    self.in_channels=out_channels
    for i in range(1,num_blocks):
      layers.append(block(self.in_channels,out_channels,0.3,1))
      
    return nn.Sequential(*layers)
  
  def forward(self,x):
    out = self.conv1(x)
    out = self.stage1(out)
    out = self.stage2(out)
    out = self.stage3(out)
    #out = self.lastact(out)
    out = self.avgPool(out)
    out = out.view(out.size(0), -1)
    #added softmax at the end to calculate direct probabilites
    return self.classifier(out)

In [5]:
sample_data = torch.ones(12, 16,32,32)
sample_input = Variable(sample_data)
BasicBlock(16,16,0.3)(sample_input).shape

torch.Size([12, 16, 32, 32])

In [7]:
sample_data = torch.ones(12,3,64,64)
sample_input = Variable(sample_data)
result = Resnet(BasicBlock,[3,3,3])(sample_input)
result.shape

torch.Size([12, 200])

In [9]:
Resnet(BasicBlock,[3,3,3])

Resnet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (stage1): Sequential(
    (0): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (drop): Dropout(p=0.3, inplace)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace)
    )
    (1): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (drop): Dropout(p=0.3, inplace)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 